In [1]:
%matplotlib inline
from __future__ import unicode_literals
import re
from scipy import ndimage
import cairocffi as cairo
import numpy as np
import random
from keras.preprocessing import image
from matplotlib import pyplot as plt
from matplotlib.font_manager import FontProperties

random.seed(55)
np.random.seed(55)

# this creates larger "blotches" of noise which look
# more realistic than just adding gaussian noise
# assumes greyscale with pixels ranging from 0 to 1

def speckle(img):
    severity = np.random.uniform(0, 0.6)
    blur = ndimage.gaussian_filter(np.random.randn(*img.shape) * severity, 1)
    img_speck = (img + blur)
    img_speck[img_speck > 1] = 1
    img_speck[img_speck <= 0] = 0
    return img_speck


# paints the string in a random location the bounding box
# also uses a random font, a slight random rotation,
# and a random amount of speckle noise

def paint_text(text, w, h, rotate=False, ud=False, multi_fonts=False, multi_sizes=False, save=False):
    surface = cairo.ImageSurface(cairo.FORMAT_RGB24, w, h)
    with cairo.Context(surface) as context:
        context.set_source_rgb(1, 1, 1)  # White
        context.paint()
        # this font list works in Centos 7
        if multi_fonts:
            fonts = ['FreeMono','Serif','FreeSerif','FreeSans','BPG Glakho','Monospace']
            context.select_font_face(np.random.choice(fonts), cairo.FONT_SLANT_NORMAL,
                                     np.random.choice([cairo.FONT_WEIGHT_BOLD, cairo.FONT_WEIGHT_NORMAL]))
        else:
            context.select_font_face('Courier', cairo.FONT_SLANT_NORMAL, cairo.FONT_WEIGHT_BOLD)
        if (multi_sizes):
          context.set_font_size(random.randint(14, 32))
        else: 
          context.set_font_size(25)
        box = context.text_extents(text)
        border_w_h = (4, 4)
        if box[2] > (w - 2 * border_w_h[1]) or box[3] > (h - 2 * border_w_h[0]):
            raise IOError('Could not fit string into image. Max char count is too large for given image width.')

        # teach the RNN translational invariance by
        # fitting text box randomly on canvas, with some room to rotate
        max_shift_x = w - box[2] - border_w_h[0]
        max_shift_y = h - box[3] - border_w_h[1]
        top_left_x = np.random.randint(0, int(max_shift_x))
        if ud:
            top_left_y = np.random.randint(0, int(max_shift_y))
        else:
            top_left_y = h // 2
        context.move_to(top_left_x - int(box[0]), top_left_y - int(box[1]))
        context.set_source_rgb(0, 0, 0)
        context.show_text(text)
    if save:
      surface.write_to_png('img_'+text+'.png')
    buf = surface.get_data()
    a = np.frombuffer(buf, np.uint8)
    a.shape = (h, w, 4)
    a = a[:, :, 0]  # grab single channel
    a = a.astype(np.float32) / 255
    a = np.expand_dims(a, 0)
    if rotate:
        a = image.random_rotation(a, 3 * (w - top_left_x) / w + 1)
    a = speckle(a)
    return a
   


Using TensorFlow backend.


In [6]:
#latin_upper = u'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
#latin_lower = u'abcdefghijklmnopqrstuvwxyz'
georgian = u'აბგდევზთიკლმნოპჟრსტუფქღყშჩცძწჭხჯჰჱჲჳჴჵ'
numbers = u'1234567890'
symbols = u'!@#$%^&*()-+=/\\.,<>?;:"|}{[]'

chars = georgian + numbers + symbols
size = len(chars)
fig = plt.figure(figsize=(8, 8))
x_train = np.zeros([60000,64,64])
y_train = [' '] * 60000

for i in range(60000):
    char = chars[random.randint(0,size-1)]    
    img = paint_text(char,64,64,save=False, rotate=True, ud=True, multi_fonts=True, multi_sizes=True)
    x_train[i] = 1-img
    y_train[i] = char
x_train 


In [10]:
1-x_train[1]

array([[  3.30916604e-04,   9.80060178e-05,   0.00000000e+00, ...,
          1.38396618e-04,   1.46452310e-04,   8.62316428e-05],
       [  2.25537026e-04,   6.38019507e-05,   0.00000000e+00, ...,
          1.17819927e-04,   9.02767962e-05,   0.00000000e+00],
       [  9.89784078e-05,   1.26512742e-05,   0.00000000e+00, ...,
          1.12573316e-04,   1.05608554e-04,   7.08971025e-05],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   2.30979898e-07, ...,
          1.99457013e-04,   8.66401396e-05,   3.38707422e-05],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.58737524e-04,   7.63362309e-05,   8.49035337e-05],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          2.30683670e-05,   0.00000000e+00,   0.00000000e+00]])